In [1]:
from lxml import html

# from bs4 import BeautifulSoup
from bs4 import BeautifulSoup, NavigableString
import time
import datetime
import threading
from apscheduler.schedulers.background import BackgroundScheduler

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec

import json
import os
import sys

import helper

from selenium.common.exceptions import TimeoutException

In [2]:
course_schedule = []
NO_WINDOW = True
IMPLICIT_WAIT_TIME = 20
EXPLICIT_WAIT_TIME = 20
WINDOWS_GRID_X = 3
WINDOWS_GRID_Y = 2
# SCREEN_WIDTH = 2560
# SCREEN_HEIGHT = 1440
# SCREEN_WIDTH = 2048
# SCREEN_HEIGHT = 1152
SCREEN_WIDTH = 1920
SCREEN_HEIGHT = 1200
# SCREEN_WIDTH = 3360
# SCREEN_HEIGHT = 1890
WINDOW_WIDTH = SCREEN_WIDTH / WINDOWS_GRID_X
WINDOW_HEIGHT = SCREEN_HEIGHT / WINDOWS_GRID_Y

In [3]:
def create_new_driver():
    #setup option for chrome profile
    # chrome_options = Options()
    # chrome_options.add_argument("user-data-dir=/Users/huohsien/Library/Application Support/Google/Chrome/Default/")
    options = Options()
    
    if NO_WINDOW:
        options = Options()
        options.add_argument('--headless')
        options.add_argument('--no-sandbox')
        options.add_argument('--disable-gpu')

    #start web driver
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    driver.implicitly_wait(IMPLICIT_WAIT_TIME)

    wait = WebDriverWait(driver, EXPLICIT_WAIT_TIME)
    return driver, wait

In [4]:
def send_keys(elm, string):
    for c in string:
        elm.send_keys(c)

In [5]:
def sync_get_element_by_xpath(driver, xpath):
    wait = WebDriverWait(driver, 15)
    try:
        wait.until(ec.visibility_of_element_located((By.XPATH, xpath)))
    except TimeoutException:
#         print("Get element from xpath:{} Timeout".format(xpath))
        return None
    return driver.find_element_by_xpath(xpath)


In [6]:
def sign_in(driver, wait):
    
    
    username = driver.find_element_by_id('ctl00_cphContents_txtUsername')
    password = driver.find_element_by_id('ctl00_cphContents_txtPassword')
    username.send_keys("DN20092360")
    password.send_keys("jj1216")
#     send_keys(username, "DN20092360")
#     send_keys(password, "jj1216")

    wait.until(ec.element_to_be_clickable((By.CLASS_NAME, "submit-button")))
#     wait.until(ec.presence_of_element_located((By.XPATH, "//div[@class='submit-button']/input")))

    sign_in_btn = driver.find_element_by_class_name("submit-button") 
    sign_in_btn.click()      
        

def log_out(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Logout')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
    
def click_book_for_class(driver, wait):
    click_btn = helper.sync_get_element_by_xpath(driver, "//a[contains(text(), 'Book for Class')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()

def click_book_this_class_now(driver, wait):
    click_btn = sync_get_element_by_xpath(driver, "//a[contains(text(), 'BOOK THIS CLASS NOW')]")
#     print("click_btn: ", click_btn)
    if click_btn:
        click_btn.click()
        return True
    else:
        return False

In [7]:
#launch url
base_url = "https://www.trueclassbooking.com.tw/member/search-class.aspx"

In [8]:
with open(r'./class_schedules/01 March - 07 March 2021.json', "r") as read_file:
    course_schedule = json.load(read_file)

In [9]:
# course_schedule

In [10]:
# NUM_CLASSES = 1
# DELAY_BETWEEN_CLASSSES = 3
# BASE_DELAY = 5
# TEST_SIGNIN_SECONDS_DELAY_FROM_NOW = BASE_DELAY
# TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW =  (NUM_CLASSES-1) * DELAY_BETWEEN_CLASSSES + BASE_DELAY*2

# signin_datetime = datetime.datetime.now() + datetime.timedelta(seconds=TEST_SIGNIN_SECONDS_DELAY_FROM_NOW)
# t = datetime.datetime.now()
# future_now = datetime.datetime(t.year, t.month, t.day, t.hour, t.minute + 1, 0)
# book_datetime = future_now + datetime.timedelta(seconds=TEST_BOOK_CLASSES_SECONDS_DELAY_FROM_NOW)

# print("Now: {}".format(datetime.datetime.now()))
# print("book_datetime: {}".format(book_datetime))
# courses_to_be_booked = []

# c = helper.search_courses_by_date('23', course_schedule)
# c = helper.search_courses_by_name('Mysore', c)
# # c = helper.search_courses_by_time('AM', c)


# courses_to_be_booked.extend(c)

# helper.list_courses_to_be_booked(courses_to_be_booked)

In [11]:
import datetime
import pytz

timezone = pytz.timezone("Asia/Taipei")

# scheduler = BackgroundScheduler()
scheduler = BackgroundScheduler(timezone='Asia/Taipei')

In [12]:
YEAR = 2021
MONTH = 2
Day = 26

book_datetime = datetime.datetime(YEAR, MONTH, Day, 22, 0 +6, 0, tzinfo=timezone)

temp = []

# c = helper.search_courses_by_date('1', course_schedule)
# c = helper.search_courses_by_name('Mysore', c)
# c = helper.search_courses_by_teacher('Simon', c)
# c = helper.search_courses_by_time('AM', c)
# temp.extend(c)

# # Monday
# #
# c = helper.search_courses_by_date('1', course_schedule)
# # c = helper.search_courses_by_name('Core', c)
# c = helper.search_courses_by_teacher('Ethan', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('1', course_schedule)
# # c = helper.search_courses_by_name('Hot', c)
# c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# Tuesday
#
c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('Ha', c)
c = helper.search_courses_by_teacher('Mei', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('War', c)
c = helper.search_courses_by_teacher('Mei', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('Core', c)
c = helper.search_courses_by_teacher('Mei', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('Intr', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('War', c)
c = helper.search_courses_by_teacher('Una', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)

c = helper.search_courses_by_date('2', course_schedule)
c = helper.search_courses_by_name('Ha', c)
c = helper.search_courses_by_teacher('Tumai', c)
# c = helper.search_courses_by_time('PM', c)
temp.extend(c)
#

# # Wednesday 
# #

# c = helper.search_courses_by_date('24', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Jun', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# # Thursday
# #

# c = helper.search_courses_by_date('25', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Mei', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('25', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Friday
# #

# c = helper.search_courses_by_date('26', course_schedule)
# c = helper.search_courses_by_name('Asht', c)
# c = helper.search_courses_by_teacher('Simon', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('26', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Andy', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('26', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Andy', c)
# # c = helper.search_courses_by_time('', c)
# temp.extend(c)

# c = helper.search_courses_by_date('26', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('08', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #

# # Saturday
# #

# c = helper.search_courses_by_date('27', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Simon', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('27', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('Anita', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# # Sunday
# #
# c = helper.search_courses_by_date('28', course_schedule)
# # c = helper.search_courses_by_name('', c)
# c = helper.search_courses_by_teacher('YaYa', c)
# c = helper.search_courses_by_time('PM', c)
# temp.extend(c)


# c = helper.search_courses_by_date('28', course_schedule)
# c = helper.search_courses_by_name('Warm', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('28', course_schedule)
# c = helper.search_courses_by_name('Vin', c)
# c = helper.search_courses_by_teacher('Una', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)

# c = helper.search_courses_by_date('28', course_schedule)
# c = helper.search_courses_by_name('初', c)
# c = helper.search_courses_by_teacher('Ani', c)
# # c = helper.search_courses_by_time('PM', c)
# temp.extend(c)
# #


# c = helper.search_courses_by_date('14', course_schedule)
# c = helper.search_courses_by_name('Hatha', c)
# c = helper.search_courses_by_teacher('Andy', c)
# # c = helper.search_courses_by_time('', c)
# temp.extend(c)



courses_to_be_booked = temp

# courses_to_be_booked = []
# courses_to_be_booked.append(temp[3])
# courses_to_be_booked.append(temp[0])
# courses_to_be_booked.append(temp[1])   
# courses_to_be_booked.append(temp[2])

# courses_to_be_booked = []

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=747149'
# course['name'] = '艾楊格 壁繩'
# course['teacher'] = 'Yvette'
# course['date'] = '19 Oct Mon'
# course['time'] = '07:15 - 08:45 PM'
# courses_to_be_booked.append(course)

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=747148'
# course['name'] = 'Iyengar Basic 艾揚格基礎'
# course['teacher'] = 'Yvette'
# course['date'] = '19 Oct Mon'
# course['time'] = '06:00 - 07:00 PM'
# courses_to_be_booked.append(course)

# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=744153'
# course['name'] = 'Universal 1'
# course['teacher'] = 'YaYa'
# course['date'] = '18 Oct Sun'
# course['time'] = '02:00 - 03:30 PM'
# courses_to_be_booked.append(course)

# course = {}
# course['href'] = 'https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=744154'
# course['name'] = 'Yin Yoga 陰瑜珈'
# course['teacher'] = 'YaYa'
# course['date'] = '18 Oct Sun'
# course['time'] = '03:45 - 04:45 PM'
# courses_to_be_booked.append(course)

helper.list_courses_to_be_booked(courses_to_be_booked)

# Set Sign In time 10 minutes before the open for booking time
signin_datetime = book_datetime - datetime.timedelta(seconds=600)

Date: 02 Mar Tue
-----------------------

Name: Hatha Yoga 哈達瑜珈 1
Teacher: Mei Wang
Time: 03:15 - 04:15 PM


Name: Warm Stretch 溫和伸展
Teacher: Mei Wang
Time: 12:15 - 01:15 PM


Name: Core Yoga 核心瑜珈
Teacher: Mei Wang
Time: 02:00 - 03:00 PM


Name: Vinyasa Intro 流暢瑜珈入門
Teacher: Una
Time: 06:00 - 07:00 PM


Name: Warm Stretch 溫和伸展
Teacher: Una
Time: 04:00 - 05:00 PM


Name: Hatha Yoga 哈達瑜珈
Teacher: Tumai
Time: 07:15 - 08:15 PM




In [13]:
book_datetime = book_datetime + datetime.timedelta(seconds=0.35)

In [14]:
def scheduled_sign_in(driver, wait):
#     print("scheduled_sign_in({},{})".format(driver, wait))
    driver.get(base_url)
    sign_in(driver, wait)

In [15]:
reserve_class_log_dicts = []

def reserve_class(driver, wait, course_to_be_booked):
    
    print("reserve_class: ({})\n".format(course_to_be_booked['time']))
    
    book_url = course_to_be_booked['href']
    class_name = course_to_be_booked['name']
    
    reserve_class_log_dict = {}
    
    reserve_class_log_dict['name'] = class_name
    
    now1 = datetime.datetime.now()
    driver.get(book_url)
    now2 = datetime.datetime.now()

    now3 = datetime.datetime.now()
    now4 = datetime.datetime.now()
    if click_book_this_class_now(driver, wait):
        now4 = datetime.datetime.now()
    else:
        now4 = datetime.datetime.now()

    reserve_class_log_dict['datetime1'] = now1
    reserve_class_log_dict['datetime2'] = now2
    reserve_class_log_dict['datetime3'] = now3
    reserve_class_log_dict['datetime4'] = now4
    
    reserve_class_log_dicts.append(reserve_class_log_dict)
    

In [16]:
print(datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))

20210226_214417


In [17]:
def timestamp_string():

    date_time = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     date_time = datetime.datetime.now().strftime("%S")

    return date_time

log_file_name = "./log_booking_a_class-- {}.csv".format(timestamp_string())


def log_message(message_str):
    print(message_str)
    with open(log_file_name, 'a') as write_file:
        write_file.write("{}\n".format(message_str))

In [18]:
def show_all_booking_result():
    log_message("Scheduling Logs:")
    
    for reserve_class_log_dict in reserve_class_log_dicts:

        log_message("Class: {}".format(reserve_class_log_dict['name']))
        log_message("-----------------------------------------\n")
        log_message("\t** Begin: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime1']))
        log_message("\t** End: Go to the class page at: {}\n".format(reserve_class_log_dict['datetime2']))

        log_message("\t**Begin: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime3']))
        log_message("\t**End: Click 'BOOK THIS CLASS NOW' button at: {}\n".format(reserve_class_log_dict['datetime4']))
        log_message("-----------------------------------------\n")

In [19]:
drivers = []
waits = []

start = time.time()
for idx, course_to_be_booked in enumerate(courses_to_be_booked):
    
    driver, wait = create_new_driver()
    driver.set_window_position(WINDOW_WIDTH * (idx % 3), WINDOW_HEIGHT * int(idx / 3));
    driver.set_window_size(WINDOW_WIDTH, WINDOW_HEIGHT)
    signin_datetime_var = signin_datetime + datetime.timedelta(seconds=(idx+1)*1)
    print("idx: ", idx, "scheduled signin datetime: ", signin_datetime_var)
    print("idx: ", idx, "scheduled book datetime: ", book_datetime)
    
    scheduler.add_job(scheduled_sign_in, 'date', run_date=signin_datetime_var, args=[driver, wait])
    scheduler.add_job(reserve_class, 'date', run_date= book_datetime, args=[driver, wait, course_to_be_booked])
    
    if idx == len(courses_to_be_booked)-1:
        show_log_time =  book_datetime + datetime.timedelta(seconds=6)
        print("show log time:{}".format(show_log_time))
        scheduler.add_job(show_all_booking_result, 'date', run_date=show_log_time)

# print("Time Elapsed: {:.2f}".format(time.time() - start)) 
scheduler.start()
 


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache
[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


idx:  0 scheduled signin datetime:  2021-02-26 21:56:01+08:06
idx:  0 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06




[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


idx:  1 scheduled signin datetime:  2021-02-26 21:56:02+08:06
idx:  1 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06




...

idx:  2 scheduled signin datetime:  2021-02-26 21:56:03+08:06
idx:  2 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06




[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


idx:  3 scheduled signin datetime:  2021-02-26 21:56:04+08:06
idx:  3 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06




[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/home/huohsien/.wdm/drivers/chromedriver/linux64/87.0.4280.88/chromedriver] found in cache


idx:  4 scheduled signin datetime:  2021-02-26 21:56:05+08:06
idx:  4 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06


idx:  5 scheduled signin datetime:  2021-02-26 21:56:06+08:06
idx:  5 scheduled book datetime:  2021-02-26 22:06:00.350000+08:06
show log time:2021-02-26 22:06:06.350000+08:06


In [20]:
###### print("https://www.trueclassbooking.com.tw/member/login.aspx\n")
for idx, course_to_be_booked in enumerate(courses_to_be_booked):
    print("Class: {} \n\turl: {}\n".format(course_to_be_booked['name'], course_to_be_booked['href']))

Class: Hatha Yoga 哈達瑜珈 1 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794625

Class: Warm Stretch 溫和伸展 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794662

Class: Core Yoga 核心瑜珈 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794663

Class: Vinyasa Intro 流暢瑜珈入門 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794627

Class: Warm Stretch 溫和伸展 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794664

Class: Hatha Yoga 哈達瑜珈 
	url: https://www.trueclassbooking.com.tw/member/popup/class-info.aspx?tcl_id=794628

